In [1]:
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
df_new_sign_up_20200314=pd.read_csv("/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-14/MediaStormMasterWeekly20200317-112739-745.txt",
                                   dtype=str,sep="|")
df_new_sign_up_20200314.shape

(115612, 13)

In [3]:
df_new_sign_up_20200314.head(2)

,customer_id_hashed,email_address_hash,sign_up_date,sign_up_channel,sign_up_location,customer_zip_code,transaction_count,transaction_amount,experian_multi_cluster,experian_demo_cluster,purchase_channel,email_unsubscribe_indicator,email_undeliverable_indicator
0,4566f5d8a70872c8293384ed5a7a9a8333aa86e592973b...,82abc4ed6676a6bedbe8c6f72a0b4152bbdf5dff77e2e3...,2020-03-10,ONLINE,6990,NaN,000000000000000.,0000000000000000.00,0,0,NONE,N,N
1,3b8b26a4645b69208f2ad785ae3966ed295695a6eb8e41...,b8915d7e98944691dfc21b3c1c42efd0376afd3c27c063...,2020-03-13,STORE,397,23518,000000000000001.,0000000000000050.98,0,0,STORES,N,N


In [4]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [10]:
list_pos_files=list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/"))
list_pos_files=[x for x in list_pos_files if "dailysales" in x.lower()]
list_pos_files=[x for x in list_pos_files if x.split("/MediaStorm_")[1][:10]>="2020-03-14"]
list_pos_files

['/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-28/MediaStormDailySales20200331-112303-873.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-14/MediaStormDailySales20200317-112349-402.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-21/MediaStormDailySales20200324-112302-496.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-04-04/MediaStormDailySales20200407-110641-771.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-04-11/MediaStormDailySales20200414-120340-722.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-04-18/MediaStormDailySales20200421-111910-356.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-04-25/MediaStormDailySales20200428-111922-148.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-05-02/MediaStormDailySales20200505-111805-965.txt']

In [14]:
df_all_sales=pd.DataFrame()
for file in list_pos_files:
    df=pd.read_csv(file,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df.sort_values(['customer_id_hashed','transaction_dt','transaction_id','location_id'])
    df_all_sales=df_all_sales.append(df)
df_all_sales=df_all_sales.drop_duplicates("customer_id_hashed")

In [20]:
df_first_act=df_all_sales[['customer_id_hashed','location_id']]
df_first_act.shape,df_first_act['customer_id_hashed'].nunique()
df_first_act['activiation_store']=np.where(df_first_act['location_id']==6990,"Online","Instore")

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
df_new_sign_up_20200314=pd.merge(df_new_sign_up_20200314,df_first_act,on="customer_id_hashed",how="left")
df_new_sign_up_20200314['activiation_store']=df_new_sign_up_20200314['activiation_store'].fillna("no_purchase")

In [24]:
df_output=df_new_sign_up_20200314.groupby(["sign_up_channel","activiation_store"])['customer_id_hashed'].nunique().to_frame().reset_index()
df_output

,sign_up_channel,activiation_store,customer_id_hashed
0,ONLINE,Instore,5792
1,ONLINE,Online,362
2,ONLINE,no_purchase,35591
3,STORE,Instore,69671
4,STORE,Online,20
5,STORE,no_purchase,4175


In [27]:
69671+20

69691